# SciTeX DSP Server - Signal Processing Examples

This notebook demonstrates the digital signal processing capabilities of the SciTeX DSP MCP server.

## Overview

The DSP server provides advanced signal processing tools:
- Filtering and spectral analysis
- Phase-amplitude coupling (PAC)
- Wavelet transforms
- Hilbert transforms
- Signal quality metrics

## Example 1: Basic Signal Filtering

In [ ]:
# Standard scipy filtering
standard_filtering = """
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

# Generate noisy signal
fs = 1000  # Sampling frequency
t = np.linspace(0, 1, fs)
clean_signal = np.sin(2*np.pi*50*t) + 0.5*np.sin(2*np.pi*120*t)
noise = 0.2 * np.random.randn(len(t))
noisy_signal = clean_signal + noise

# Design butterworth filter
nyquist = fs / 2
low_cutoff = 40 / nyquist
high_cutoff = 60 / nyquist
b, a = signal.butter(4, [low_cutoff, high_cutoff], btype='band')

# Apply filter
filtered = signal.filtfilt(b, a, noisy_signal)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(t[:200], noisy_signal[:200], alpha=0.7, label='Noisy')
plt.plot(t[:200], filtered[:200], label='Filtered')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.title('Butterworth Bandpass Filter')
"""

print("STANDARD SCIPY FILTERING:")
print(standard_filtering)

In [ ]:
# SciTeX enhanced filtering
scitex_filtering = '''
import scitex as stx
import numpy as np

def advanced_filtering_demo():
    """Demonstrate advanced filtering capabilities."""
    # Generate test signal with multiple components
    fs = 1000  # Sampling frequency
    t = np.linspace(0, 2, 2*fs)
    
    # Signal components
    theta = np.sin(2*np.pi*6*t)      # Theta rhythm (6 Hz)
    gamma = 0.3*np.sin(2*np.pi*50*t) # Gamma rhythm (50 Hz)
    noise = 0.2*np.random.randn(len(t))
    signal = theta + gamma + noise
    
    # Apply multiple filters with automatic parameter selection
    filters = stx.dsp.create_filter_bank(
        fs=fs,
        bands={
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30),
            'gamma': (30, 100)
        },
        filter_type='butterworth',
        order='auto',  # Automatically select optimal order
        method='zero-phase'  # Use filtfilt for zero phase distortion
    )
    
    # Apply filters and analyze
    filtered_signals = stx.dsp.apply_filter_bank(
        signal, 
        filters,
        return_dict=True,
        compute_envelope=True,
        compute_phase=True
    )
    
    # Adaptive filtering for artifact removal
    cleaned_signal = stx.dsp.adaptive_filter(
        signal,
        reference_noise=noise,
        method='nlms',  # Normalized LMS
        step_size='auto',
        filter_length=32
    )
    
    # Create comprehensive visualization
    fig = stx.plt.create_dsp_figure(
        n_panels=4,
        figsize=(12, 10)
    )
    
    # Panel 1: Original and cleaned signal
    fig.panels[0].plot_signals(
        [signal, cleaned_signal],
        labels=['Original', 'Adaptive Filtered'],
        time_axis=t,
        title='Signal Cleaning'
    )
    
    # Panel 2: Filter bank outputs
    fig.panels[1].plot_filter_bank_output(
        filtered_signals,
        time_axis=t,
        show_envelope=True,
        title='Frequency Band Decomposition'
    )
    
    # Panel 3: Power spectral density
    fig.panels[2].plot_psd_comparison(
        {'Original': signal, 'Cleaned': cleaned_signal},
        fs=fs,
        method='welch',
        show_bands=True,
        log_scale=True,
        title='Power Spectral Density'
    )
    
    # Panel 4: Filter frequency responses
    fig.panels[3].plot_filter_responses(
        filters,
        fs=fs,
        show_phase=True,
        show_group_delay=True,
        title='Filter Characteristics'
    )
    
    # Save results
    stx.io.save(fig, './figures/filtering_analysis.png',
                symlink_from_cwd=True)
    
    # Export filter specifications
    stx.io.save(filters, './filters/filter_bank_specs.json',
                symlink_from_cwd=True)
    
    return filtered_signals, filters
'''

print("SCITEX ENHANCED FILTERING:")
print(scitex_filtering)

## Example 2: Phase-Amplitude Coupling (PAC) Analysis

In [ ]:
# PAC analysis with SciTeX
pac_analysis = '''
import scitex as stx
import numpy as np

def analyze_phase_amplitude_coupling():
    """Analyze cross-frequency coupling in neural signals."""
    # Load or generate signal
    fs = 1000  # Sampling rate
    duration = 10  # seconds
    t = np.linspace(0, duration, int(fs*duration))
    
    # Generate synthetic signal with PAC
    # Low frequency phase modulates high frequency amplitude
    phase_freq = 6  # Hz (theta)
    amp_freq = 50   # Hz (gamma)
    
    phase_signal = np.sin(2*np.pi*phase_freq*t)
    modulation = 0.5 * (1 + np.sin(2*np.pi*phase_freq*t))
    amp_signal = modulation * np.sin(2*np.pi*amp_freq*t)
    signal = phase_signal + amp_signal + 0.1*np.random.randn(len(t))
    
    # Comprehensive PAC analysis
    pac_results = stx.dsp.compute_pac(
        signal,
        fs=fs,
        phase_freqs=np.arange(2, 20, 1),    # 2-20 Hz for phase
        amp_freqs=np.arange(30, 100, 5),    # 30-100 Hz for amplitude
        method='tort',  # Options: 'tort', 'canolty', 'ozkurt', 'plv'
        n_surrogates=200,  # For statistical testing
        correct_for_multiple_comparisons=True,
        n_jobs=-1  # Parallel processing
    )
    
    # Time-resolved PAC
    time_pac = stx.dsp.compute_pac_time(
        signal,
        fs=fs,
        phase_freq_band=(4, 8),   # Theta
        amp_freq_band=(30, 80),   # Gamma
        window_size=2.0,  # seconds
        step_size=0.1,    # seconds
        method='mvl'      # Mean vector length
    )
    
    # Phase-specific analysis
    phase_analysis = stx.dsp.phase_triggered_average(
        signal,
        fs=fs,
        trigger_freq=phase_freq,
        n_bins=18,  # 20-degree bins
        analyze_freqs=np.arange(30, 100, 5)
    )
    
    # Create comprehensive PAC figure
    fig = stx.plt.create_pac_figure(
        layout='comprehensive',
        figsize=(15, 12)
    )
    
    # Panel 1: Comodulogram
    fig.panels['comodulogram'].plot_pac_comodulogram(
        pac_results['mi_matrix'],
        phase_freqs=pac_results['phase_freqs'],
        amp_freqs=pac_results['amp_freqs'],
        significance_mask=pac_results['significant'],
        cmap='hot',
        title='Phase-Amplitude Coupling Strength'
    )
    
    # Panel 2: Time-resolved PAC
    fig.panels['time_pac'].plot_pac_time(
        time_pac['time'],
        time_pac['pac_values'],
        confidence_intervals=time_pac['ci'],
        events=None,  # Can add event markers
        title='PAC Over Time'
    )
    
    # Panel 3: Phase-amplitude distribution
    fig.panels['phase_dist'].plot_phase_amplitude_dist(
        phase_analysis['phase_bins'],
        phase_analysis['amplitude_by_phase'],
        circular=True,
        show_preferred_phase=True,
        title='Amplitude Distribution by Phase'
    )
    
    # Panel 4: Example signal segment
    fig.panels['signal'].plot_pac_example(
        t[:2000], signal[:2000],
        phase_signal=phase_signal[:2000],
        amplitude_envelope=amp_signal[:2000],
        title='Signal Example with PAC'
    )
    
    # Statistical summary
    fig.panels['stats'].add_pac_statistics(
        pac_results,
        include_surrogate_dist=True,
        include_effect_size=True
    )
    
    # Save comprehensive results
    stx.io.save(fig, './figures/pac_analysis.png',
                dpi=300, symlink_from_cwd=True)
    
    stx.io.save(pac_results, './results/pac_full_results.npz',
                compress=True, symlink_from_cwd=True)
    
    # Generate report
    report = stx.dsp.generate_pac_report(
        pac_results,
        time_pac,
        signal_info={'fs': fs, 'duration': duration},
        format='markdown'
    )
    
    stx.io.save(report, './reports/pac_analysis_report.md',
                symlink_from_cwd=True)
    
    return pac_results
'''

print("PHASE-AMPLITUDE COUPLING ANALYSIS:")
print(pac_analysis)

## Example 3: Wavelet Analysis

In [ ]:
# Wavelet transform analysis
wavelet_analysis = '''
import scitex as stx
import numpy as np

def wavelet_time_frequency_analysis():
    """Perform wavelet-based time-frequency analysis."""
    # Generate chirp signal (frequency increases over time)
    fs = 1000
    t = np.linspace(0, 5, 5*fs)
    f0, f1 = 10, 50  # Start and end frequencies
    chirp = np.sin(2*np.pi*(f0 + (f1-f0)*t/5)*t)
    
    # Add transient events
    events = [1.0, 2.5, 4.0]
    for event_time in events:
        idx = int(event_time * fs)
        chirp[idx:idx+50] += 2*np.exp(-np.linspace(0, 5, 50))
    
    # Add noise
    signal = chirp + 0.3*np.random.randn(len(t))
    
    # Continuous wavelet transform
    cwt_result = stx.dsp.compute_cwt(
        signal,
        fs=fs,
        wavelet='morlet',  # Options: 'morlet', 'mexican_hat', 'paul', 'dog'
        frequencies=np.logspace(0, 2, 100),  # 1-100 Hz
        normalize='energy',  # Preserve energy across scales
        compute_coi=True,    # Cone of influence
        n_jobs=-1
    )
    
    # Discrete wavelet transform for denoising
    dwt_result = stx.dsp.wavelet_denoise(
        signal,
        wavelet='db4',  # Daubechies 4
        level='auto',   # Automatic level selection
        threshold='soft',
        threshold_method='bayes',  # Bayesian threshold
        return_coefficients=True
    )
    
    # Wavelet packet decomposition
    wp_result = stx.dsp.wavelet_packet_analysis(
        signal,
        wavelet='db8',
        maxlevel=5,
        criterion='entropy',  # For best basis selection
        return_tree=True
    )
    
    # Ridge extraction from scalogram
    ridges = stx.dsp.extract_wavelet_ridges(
        cwt_result['scalogram'],
        cwt_result['frequencies'],
        n_ridges=3,  # Extract top 3 ridges
        min_length=0.5,  # Minimum 0.5 seconds
        gap_tolerance=0.1  # Allow 0.1s gaps
    )
    
    # Create wavelet analysis figure
    fig = stx.plt.create_wavelet_figure(
        n_panels=5,
        figsize=(14, 12)
    )
    
    # Panel 1: Original signal
    ax1 = fig.panels[0]
    ax1.plot(t, signal, 'b-', alpha=0.7, label='Noisy')
    ax1.plot(t, dwt_result['denoised'], 'r-', label='Denoised')
    for event in events:
        ax1.axvline(event, color='k', linestyle='--', alpha=0.5)
    ax1.set_xyt('Time (s)', 'Amplitude', 'Signal and Wavelet Denoising')
    ax1.legend()
    
    # Panel 2: Scalogram
    ax2 = fig.panels[1]
    im = ax2.plot_scalogram(
        cwt_result['scalogram'],
        time=t,
        frequencies=cwt_result['frequencies'],
        coi=cwt_result['coi'],
        power_scale='log',
        cmap='jet',
        title='Continuous Wavelet Transform'
    )
    
    # Overlay ridges
    for ridge in ridges:
        ax2.plot(ridge['time'], ridge['frequency'], 'w-', linewidth=2)
    
    # Panel 3: DWT coefficients
    ax3 = fig.panels[2]
    ax3.plot_dwt_coefficients(
        dwt_result['coefficients'],
        wavelet=dwt_result['wavelet'],
        show_threshold=True,
        title='DWT Coefficients and Thresholding'
    )
    
    # Panel 4: Wavelet packet tree
    ax4 = fig.panels[3]
    ax4.plot_wavelet_packet_tree(
        wp_result['tree'],
        show_best_basis=True,
        color_by_energy=True,
        title='Wavelet Packet Decomposition'
    )
    
    # Panel 5: Instantaneous frequency from ridges
    ax5 = fig.panels[4]
    for i, ridge in enumerate(ridges):
        ax5.plot(ridge['time'], ridge['frequency'], 
                label=f'Component {i+1}', linewidth=2)
    ax5.set_xyt('Time (s)', 'Frequency (Hz)', 'Extracted Frequency Components')
    ax5.legend()
    ax5.set_ylim(0, 60)
    
    # Save results
    stx.io.save(fig, './figures/wavelet_analysis.png',
                dpi=300, symlink_from_cwd=True)
    
    # Export wavelet coefficients for further analysis
    wavelet_data = {
        'cwt': cwt_result,
        'dwt': dwt_result,
        'ridges': ridges,
        'signal_info': {'fs': fs, 'duration': len(t)/fs}
    }
    
    stx.io.save(wavelet_data, './results/wavelet_analysis.npz',
                compress=True, symlink_from_cwd=True)
    
    return wavelet_data
'''

print("WAVELET TIME-FREQUENCY ANALYSIS:")
print(wavelet_analysis)

## Example 4: Hilbert Transform and Analytical Signal

In [ ]:
# Hilbert transform analysis
hilbert_analysis = '''
import scitex as stx
import numpy as np

def hilbert_analytical_signal():
    """Analyze instantaneous properties using Hilbert transform."""
    # Generate amplitude-modulated signal
    fs = 1000
    t = np.linspace(0, 3, 3*fs)
    
    # Carrier and modulation
    carrier_freq = 40  # Hz
    mod_freq = 5      # Hz
    mod_depth = 0.5
    
    amplitude = 1 + mod_depth * np.sin(2*np.pi*mod_freq*t)
    signal = amplitude * np.sin(2*np.pi*carrier_freq*t)
    
    # Add frequency modulation
    freq_mod = 10 * np.sin(2*np.pi*2*t)  # 2 Hz FM
    signal += np.sin(2*np.pi*(carrier_freq*t + freq_mod))
    
    # Compute analytical signal
    analytical = stx.dsp.hilbert_transform(
        signal,
        fs=fs,
        filter_band=(30, 50),  # Focus on carrier frequency
        filter_order=4,
        return_dict=True
    )
    
    # Extract instantaneous properties
    inst_props = stx.dsp.instantaneous_properties(
        analytical['analytical_signal'],
        fs=fs,
        unwrap_phase=True,
        smooth_frequency=True,
        freq_smoothing_window=0.05  # 50ms window
    )
    
    # Phase synchronization analysis
    # Generate second signal
    signal2 = np.sin(2*np.pi*carrier_freq*t + np.pi/4)  # Phase shifted
    
    phase_sync = stx.dsp.phase_synchronization(
        signal, signal2,
        fs=fs,
        method='plv',  # Phase locking value
        window_size=0.5,  # 500ms windows
        overlap=0.8,
        freq_bands={
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30),
            'gamma': (30, 50)
        }
    )
    
    # Amplitude-amplitude coupling
    aac = stx.dsp.amplitude_coupling(
        signal, signal2,
        fs=fs,
        freq_bands=phase_sync['freq_bands'],
        method='correlation'
    )
    
    # Create comprehensive Hilbert analysis figure
    fig, axes = stx.plt.subplots(4, 2, figsize=(14, 12))
    
    # Original signal
    axes[0, 0].plot(t[:1000], signal[:1000])
    axes[0, 0].set_xyt('Time (s)', 'Amplitude', 'Original Signal')
    
    # Analytical signal (real and imaginary)
    axes[0, 1].plot(t[:1000], np.real(analytical['analytical_signal'][:1000]), 
                    label='Real', alpha=0.7)
    axes[0, 1].plot(t[:1000], np.imag(analytical['analytical_signal'][:1000]), 
                    label='Imaginary', alpha=0.7)
    axes[0, 1].set_xyt('Time (s)', 'Amplitude', 'Analytical Signal')
    axes[0, 1].legend()
    
    # Instantaneous amplitude
    axes[1, 0].plot(t, inst_props['amplitude'])
    axes[1, 0].plot(t, amplitude, 'r--', alpha=0.5, label='True envelope')
    axes[1, 0].set_xyt('Time (s)', 'Amplitude', 'Instantaneous Amplitude')
    axes[1, 0].legend()
    
    # Instantaneous frequency
    axes[1, 1].plot(t[1:], inst_props['frequency'][1:])
    axes[1, 1].axhline(carrier_freq, color='r', linestyle='--', 
                       alpha=0.5, label='Carrier freq')
    axes[1, 1].set_xyt('Time (s)', 'Frequency (Hz)', 'Instantaneous Frequency')
    axes[1, 1].legend()
    axes[1, 1].set_ylim(20, 60)
    
    # Phase difference
    phase_diff = inst_props['phase'] - (2*np.pi*carrier_freq*t)
    axes[2, 0].plot(t, np.mod(phase_diff, 2*np.pi))
    axes[2, 0].set_xyt('Time (s)', 'Phase (rad)', 'Phase Deviation')
    
    # Phase synchronization over time
    axes[2, 1].plot_phase_sync_time(
        phase_sync['time'],
        phase_sync['plv']['gamma'],
        ylabel='PLV',
        title='Phase Locking Value (Gamma Band)'
    )
    
    # Polar plot of phase relationship
    ax_polar = plt.subplot(4, 2, 7, projection='polar')
    phase1 = np.angle(analytical['analytical_signal'])
    hist, bins = np.histogram(phase1[:1000], bins=36)
    ax_polar.bar(bins[:-1], hist, width=np.diff(bins), alpha=0.7)
    ax_polar.set_title('Phase Distribution')
    
    # Amplitude-amplitude coupling matrix
    axes[3, 1].imshow(aac['coupling_matrix'], aspect='auto', cmap='RdBu_r')
    axes[3, 1].set_xticks(range(len(aac['freq_bands'])))
    axes[3, 1].set_xticklabels(aac['freq_bands'].keys())
    axes[3, 1].set_yticks(range(len(aac['freq_bands'])))
    axes[3, 1].set_yticklabels(aac['freq_bands'].keys())
    axes[3, 1].set_title('Amplitude-Amplitude Coupling')
    
    stx.plt.tight_layout()
    
    # Save results
    stx.io.save(fig, './figures/hilbert_analysis.png',
                dpi=300, symlink_from_cwd=True)
    
    return {
        'analytical': analytical,
        'instantaneous': inst_props,
        'phase_sync': phase_sync,
        'amplitude_coupling': aac
    }
'''

print("HILBERT TRANSFORM AND ANALYTICAL SIGNAL:")
print(hilbert_analysis)

## Example 5: Signal Quality and Artifact Detection

In [ ]:
# Signal quality assessment
quality_assessment = '''
import scitex as stx
import numpy as np

def assess_signal_quality():
    """Comprehensive signal quality assessment and artifact detection."""
    # Load signal with various artifacts
    fs = 1000
    t = np.linspace(0, 10, 10*fs)
    
    # Clean signal
    clean = np.sin(2*np.pi*10*t) + 0.5*np.sin(2*np.pi*25*t)
    
    # Add artifacts
    signal = clean.copy()
    
    # Motion artifact (low frequency)
    signal[2000:2500] += 5 * np.sin(2*np.pi*0.5*t[2000:2500])
    
    # Spike artifacts
    spike_indices = [1000, 3500, 7000]
    for idx in spike_indices:
        signal[idx] += np.random.randn() * 10
    
    # Line noise (50/60 Hz)
    signal[5000:6000] += 2 * np.sin(2*np.pi*50*t[5000:6000])
    
    # Saturation/clipping
    signal[8000:8500] = np.clip(signal[8000:8500], -2, 2)
    
    # Comprehensive quality assessment
    quality = stx.dsp.assess_signal_quality(
        signal,
        fs=fs,
        metrics=[
            'snr',           # Signal-to-noise ratio
            'thd',           # Total harmonic distortion
            'spectral_entropy',
            'kurtosis',
            'zero_crossings',
            'spectral_edge_frequency',
            'band_power_ratios'
        ],
        window_size=1.0,  # 1 second windows
        overlap=0.5
    )
    
    # Artifact detection
    artifacts = stx.dsp.detect_artifacts(
        signal,
        fs=fs,
        methods=[
            'amplitude_threshold',
            'gradient_threshold',
            'spectral_anomaly',
            'kurtosis_threshold',
            'muscle_artifact',
            'eye_blink',
            'line_noise',
            'saturation'
        ],
        return_segments=True,
        merge_nearby=True,
        min_artifact_duration=0.01  # 10ms
    )
    
    # Artifact removal/correction
    cleaned = stx.dsp.remove_artifacts(
        signal,
        artifacts,
        fs=fs,
        methods={
            'spike': 'interpolation',
            'motion': 'high_pass',
            'line_noise': 'notch_filter',
            'saturation': 'cubic_spline',
            'muscle': 'ica'  # Independent component analysis
        },
        preserve_length=True
    )
    
    # Signal quality index (SQI)
    sqi = stx.dsp.compute_sqi(
        signal,
        fs=fs,
        reference_signal=clean,  # If available
        metrics_weights={
            'snr': 0.3,
            'artifact_ratio': 0.3,
            'spectral_quality': 0.2,
            'temporal_quality': 0.2
        }
    )
    
    # Create quality assessment figure
    fig = stx.plt.create_quality_figure(
        n_rows=5,
        figsize=(14, 12),
        sharex=True
    )
    
    # Row 1: Original signal with artifacts marked
    ax1 = fig.axes[0]
    ax1.plot(t, signal, 'b-', alpha=0.7, label='Original')
    
    # Mark different artifact types
    colors = plt.cm.tab10(np.linspace(0, 1, len(artifacts['types'])))
    for artifact_type, segments, color in zip(artifacts['types'], 
                                              artifacts['segments'], 
                                              colors):
        for start, end in segments:
            ax1.axvspan(t[start], t[end], alpha=0.3, color=color, 
                       label=artifact_type)
    
    ax1.set_ylabel('Amplitude')
    ax1.set_title('Signal with Detected Artifacts')
    ax1.legend(loc='upper right', ncol=4)
    
    # Row 2: Cleaned signal
    ax2 = fig.axes[1]
    ax2.plot(t, cleaned['signal'], 'g-', alpha=0.7, label='Cleaned')
    ax2.plot(t, clean, 'k--', alpha=0.5, label='Ground truth')
    ax2.set_ylabel('Amplitude')
    ax2.set_title('Artifact-Corrected Signal')
    ax2.legend()
    
    # Row 3: Quality metrics over time
    ax3 = fig.axes[2]
    ax3.plot(quality['time'], quality['snr'], label='SNR')
    ax3.set_ylabel('SNR (dB)')
    ax3.set_title('Signal-to-Noise Ratio')
    ax3_twin = ax3.twinx()
    ax3_twin.plot(quality['time'], quality['spectral_entropy'], 
                  'r-', label='Spectral Entropy')
    ax3_twin.set_ylabel('Entropy')
    
    # Row 4: Spectral quality
    ax4 = fig.axes[3]
    ax4.plot_spectrogram_diff(
        signal, cleaned['signal'],
        fs=fs,
        title='Spectral Difference (Original - Cleaned)'
    )
    
    # Row 5: SQI over time
    ax5 = fig.axes[4]
    ax5.plot(sqi['time'], sqi['overall'], 'b-', linewidth=2)
    ax5.fill_between(sqi['time'], 0, sqi['overall'], alpha=0.3)
    ax5.axhline(0.7, color='r', linestyle='--', label='Acceptable threshold')
    ax5.set_xlabel('Time (s)')
    ax5.set_ylabel('Signal Quality Index')
    ax5.set_title('Overall Signal Quality')
    ax5.set_ylim(0, 1)
    ax5.legend()
    
    # Save results
    stx.io.save(fig, './figures/signal_quality_assessment.png',
                dpi=300, symlink_from_cwd=True)
    
    # Generate quality report
    report = stx.dsp.generate_quality_report(
        quality,
        artifacts,
        sqi,
        recommendations=True
    )
    
    stx.io.save(report, './reports/signal_quality_report.html',
                format='html', symlink_from_cwd=True)
    
    return {
        'quality_metrics': quality,
        'artifacts': artifacts,
        'cleaned_signal': cleaned,
        'sqi': sqi
    }
'''

print("SIGNAL QUALITY AND ARTIFACT DETECTION:")
print(quality_assessment)

## Summary

The SciTeX DSP Server provides comprehensive signal processing capabilities:

### 1. **Advanced Filtering**
   - Automatic filter design and optimization
   - Filter banks for frequency decomposition
   - Adaptive filtering for artifact removal
   - Zero-phase filtering

### 2. **Phase-Amplitude Coupling**
   - Multiple PAC methods (Tort, Canolty, MVL, etc.)
   - Time-resolved PAC analysis
   - Statistical significance testing
   - Comprehensive visualization

### 3. **Wavelet Analysis**
   - Continuous and discrete wavelet transforms
   - Wavelet denoising with optimal thresholding
   - Ridge extraction for instantaneous frequency
   - Wavelet packet decomposition

### 4. **Hilbert Transform**
   - Analytical signal computation
   - Instantaneous amplitude, phase, and frequency
   - Phase synchronization analysis
   - Amplitude coupling measures

### 5. **Signal Quality**
   - Comprehensive quality metrics
   - Multi-method artifact detection
   - Intelligent artifact removal
   - Signal Quality Index (SQI)

## Key Features

- **Parallel Processing**: Multi-core support for computationally intensive operations
- **Automatic Parameter Selection**: Optimal parameters chosen based on signal properties
- **Statistical Validation**: Built-in significance testing and surrogate generation
- **Comprehensive Visualization**: Publication-ready figures for all analyses
- **Report Generation**: Automated analysis reports in multiple formats

## Benefits

- **Efficiency**: Complex analyses in few lines of code
- **Reproducibility**: All parameters and results saved automatically
- **Robustness**: Multiple methods with automatic selection
- **Integration**: Works seamlessly with other SciTeX modules
- **Validation**: Built-in quality checks and artifact handling